Data preparation

In [3]:
import os
import shutil
import yaml
from sklearn.model_selection import train_test_split

def convert_yolo_to_classification(source_dir, target_dir, yaml_file):
    # Read the YAML file
    with open(yaml_file, 'r') as file:
        data = yaml.safe_load(file)
    
    class_names = data['names']
    
    # Create directories for each class
    for split in ['train', 'valid', 'test']:
        for class_name in class_names:
            os.makedirs(os.path.join(target_dir, split, class_name), exist_ok=True)
    
    # Process each split
    for split in ['train', 'valid', 'test']:
        image_dir = os.path.join(source_dir, split, 'images')
        label_dir = os.path.join(source_dir, split, 'labels')
        
        for image_file in os.listdir(image_dir):
            image_path = os.path.join(image_dir, image_file)
            label_path = os.path.join(label_dir, os.path.splitext(image_file)[0] + '.txt')
            
            # Read the first class from the label file
            with open(label_path, 'r') as file:
                first_line = file.readline().strip().split()
                if first_line:
                    class_index = int(first_line[0])
                    class_name = class_names[class_index]
                    
                    # Copy the image to the appropriate class folder
                    shutil.copy(image_path, os.path.join(target_dir, split, class_name))

# Usage
source_dir = 'D://Work//coding//vuzz//new//MIREA_7_semester_VT//rsppr//SAFE-MACS//data//cppe-5-yolov10'
target_dir = 'D://Work//coding//vuzz//new//MIREA_7_semester_VT//rsppr//SAFE-MACS//data//cppe-5-eff'
yaml_file = 'D://Work//coding//vuzz//new//MIREA_7_semester_VT//rsppr//SAFE-MACS//data//cppe-5-yolov10//data.yaml'

convert_yolo_to_classification(source_dir, target_dir, yaml_file)


EfficientNet v2 Train

YOLO + EfficientNet (avg res)

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
from ultralytics import YOLO
from efficientnet_pytorch import EfficientNet

yolo_model = YOLO('yolov10-l.pt')

efficientnet_model = EfficientNet.from_pretrained('efficientnet-b0')
efficientnet_model.eval()

# Image transformations for EfficientNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def detect_and_classify(image_path):
    image = Image.open(image_path)
    
    yolo_results = yolo_model(image)
    
    final_results = []
    
    for detection in yolo_results[0].boxes.data:
        x1, y1, x2, y2, conf, class_id = detection
        
        cropped_image = image.crop((x1, y1, x2, y2))
        
        input_tensor = transform(cropped_image).unsqueeze(0)
        
        # Perform EfficientNet classification
        with torch.no_grad():
            efficientnet_output = efficientnet_model(input_tensor)
        
        # Get EfficientNet probabilities
        efficientnet_probs = torch.nn.functional.softmax(efficientnet_output[0], dim=0)
        
        # Get YOLO and EfficientNet top classes
        yolo_class = int(class_id)
        efficientnet_class = efficientnet_probs.argmax().item()
        
        # Calculate average probability
        avg_prob = (conf + efficientnet_probs[efficientnet_class]) / 2
        
        # Determine final class based on average probability
        final_class = yolo_class if conf > efficientnet_probs[efficientnet_class] else efficientnet_class
        
        final_results.append({
            'bbox': (x1, y1, x2, y2),
            'yolo_class': yolo_class,
            'yolo_conf': conf,
            'efficientnet_class': efficientnet_class,
            'efficientnet_conf': efficientnet_probs[efficientnet_class],
            'final_class': final_class,
            'avg_prob': avg_prob
        })
    
    return final_results

# Example usage
image_path = 'path/to/your/image.jpg'
results = detect_and_classify(image_path)

for result in results:
    print(f"Detected object: {result['final_class']}")
    print(f"Bounding box: {result['bbox']}")
    print(f"YOLO class: {result['yolo_class']}, confidence: {result['yolo_conf']:.2f}")
    print(f"EfficientNet class: {result['efficientnet_class']}, confidence: {result['efficientnet_conf']:.2f}")
    print(f"Average probability: {result['avg_prob']:.2f}")
    print("---")